In [1]:
import torch
import ttnn
from model import ModelArgs
from typing import Tuple

# just a super simple forward without host overhead
from ttcode.lightweightmodule import LightweightModule

2024-12-09 17:42:20.514 | DEBUG    | ttnn:<module>:82 - Initial ttnn.CONFIG:
Config{cache_path=/home/bach/.cache/ttnn,model_cache_path=/home/bach/.cache/ttnn/models,tmp_dir=/tmp/ttnn,enable_model_cache=false,enable_fast_runtime_mode=true,throw_exception_on_fallback=false,enable_logging=false,enable_graph_report=false,enable_detailed_buffer_report=false,enable_detailed_tensor_report=false,enable_comparison_mode=false,comparison_mode_pcc=0.9999,root_report_path=generated/ttnn/reports,report_name=std::nullopt,std::nullopt}
2024-12-09 17:42:20.572 | WARNING  | ttnn.decorators:operation_decorator:801 - Should ttnn.pearson_correlation_coefficient be migrated to C++?
2024-12-09 17:42:20.573 | WARNING  | ttnn.decorators:operation_decorator:801 - Should ttnn.Conv1d be migrated to C++?
2024-12-09 17:42:20.574 | WARNING  | ttnn.decorators:operation_decorator:801 - Should ttnn.conv2d be migrated to C++?
2024-12-09 17:42:20.575 | WARNING  | ttnn.decorators:operation_decorator:801 - Should ttnn.unsq

In [2]:
from dataclasses import dataclass
from typing import Optional
@dataclass
class ModelArgs:
    # default hyperparameters for the TinyStories 260k model
    dim: int = 64
    n_layers: int = 5
    n_heads: int = 8
    n_kv_heads: Optional[int] = 4
    vocab_size: int = 512
    hidden_dim: Optional[int] = None
    multiple_of: int = 4  # MLP hidden layer size will be multiple of
    norm_eps: float = 1e-5
    max_seq_len: int = 512
    dropout: float = 0.5

In [3]:
# RMS Norm imported from a common
from ttcode import rmsnorm

In [ ]:
# RoPE implementation
def precompute_freqs_cis(dim: int, end: int, theta: float = 10000.0, device=0):
    # Build the theta parameter
    # According to the formula theta_i = 10000^(-2(i-1)/dim) for i = [1, 2, ... dim/2]
    # Shape: (Head_Dim / 2)
    # freqs = 1.0 / (theta ** (ttnn.arange(0, dim, 2)[: (dim // 2)].float() / dim))
    a = torch.arange(0, dim, 2)[: (dim // 2)]
    t = ttnn.from_torch(a, dtype=ttnn.bfloat16, layout=ttnn.TILE_LAYOUT, device=device)
    e = ttnn.div(t, dim)
    # freqs = 1.0 / (theta ** e)
    freqs = 1.0 / (theta ** e)
    
    # Construct the positions (the "m" parameter)
    # Shape: (Seq_Len)
    t = ttnn.arange(end, device=freqs.device)  # type: ignore
    # Multiply each theta by each position using the outer product.
    # Shape: (Seq_Len) outer_product* (Head_Dim / 2) -> (Seq_Len, Head_Dim / 2)
    # freqs = ttnn.outer(t, freqs).float()  # type: ignore
    freqs = ttnn.outer(t, freqs)  # type: ignore
    
    # We can computer the cos and sin
    # Shape: (Seq_len, Head_Dim / 2)
    freqs_cos = ttnn.cos(freqs)  # real part
    freqs_sin = ttnn.sin(freqs)  # imaginary part
    return freqs_cos, freqs_sin

def reshape_for_broadcast(freqs_cis: ttnn.Tensor, x: ttnn.Tensor):
    ndim = x.ndim
    assert 0 <= 1 < ndim
    assert freqs_cis.shape == (x.shape[1], x.shape[-1])
    shape = [d if i == 1 or i == ndim - 1 else 1 for i, d in enumerate(x.shape)]
    return freqs_cis.view(shape)

def apply_rotary_emb(
    xq: ttnn.Tensor,
    xk: ttnn.Tensor,
    freqs_cos: ttnn.Tensor,
    freqs_sin: ttnn.Tensor
) -> Tuple[ttnn.Tensor, ttnn.Tensor]:

    # reshape xq and xk to match the complex representation
    # xq.shape[:-1] + (-1, 2) -> tolgo ultima dim, e faccio reshape che abbia come ultima dim 2
    # il -1 è per fare il modo che sia compatibile: (5,1,8) -> .reshape(5,1,-1,2) -> (5,1,4,2) 
    xq_r, xq_i = xq.float().reshape(xq.shape[:-1] + (-1, 2)).unbind(-1)
    xk_r, xk_i = xk.float().reshape(xk.shape[:-1] + (-1, 2)).unbind(-1)

    # reshape freqs_cos and freqs_sin for broadcasting
    freqs_cos = reshape_for_broadcast(freqs_cos, xq_r)
    freqs_sin = reshape_for_broadcast(freqs_sin, xq_r)

    # apply rotation using real numbers
    xq_out_r = xq_r * freqs_cos - xq_i * freqs_sin
    xq_out_i = xq_r * freqs_sin + xq_i * freqs_cos

    xk_out_r = xk_r * freqs_cos - xk_i * freqs_sin
    xk_out_i = xk_r * freqs_sin + xk_i * freqs_cos

    # flatten last two dimensions
    xq_out = torch.stack([xq_out_r, xq_out_i], dim=-1).flatten(3)
    xk_out = torch.stack([xk_out_r, xk_out_i], dim=-1).flatten(3)

    return xq_out.type_as(xq), xk_out.type_as(xk)


device = ttnn.open_device(device_id=0)
precompute_freqs_cis(4, 4, device=device)

                 Device | INFO     | Opening user mode device driver

2024-12-09 17:42:20.655 | WARNING  | SiliconDriver   - Unknown board type for chip 0. This might happen because chip is running old firmware. Defaulting to DEFAULT
2024-12-09 17:42:20.655 | WARNING  | SiliconDriver   - Unknown board type for chip 0. This might happen because chip is running old firmware. Defaulting to DEFAULT
2024-12-09 17:42:20.656 | INFO     | SiliconDriver   - Detected 1 PCI device : [0]
2024-12-09 17:42:20.656 | INFO     | SiliconDriver   - Opened PCI device 0; KMD version: 1.30.0
                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | AI CLK for device 0 is:   1000 MHz
                  Metal | INFO     | Profiler started on device 0
                 Always | WARNING  | Specifying tile shape for a row major layout is deprecated, and will be removed soon
                 Always | WARNING  | Specifying tile shape for a row major

TypeError: unsupported operand type(s) for ** or pow(): 'float' and 'ttnn._ttnn.tensor.Tensor'